## Evaluation


In [12]:
# make sure we are in WineRecommender so the paths work
import os
if 'evaluation' in os.getcwd():
    os.chdir("..")
print(os.getcwd())

/home/iris/Documents/Uni/AIR/WineRecommender


## MRR

In [13]:
import json

# Higher MRR indicates that relevant documents appear earlier in the rankings.

def calculate_MRR(model):
    ratings = scores["queries"]
    number_of_queries = len(ratings)

    total = 0
    for rating in ratings:
        rank_first_relevant = 0 # relevant: score = 3
        for wine in rating[model]: # loop over the 10 wines returned
            if wine["score"] == "2" or wine["score"] == "3":
                rank_first_relevant = wine["rank"]
                break

        if rank_first_relevant != 0:
            total += 1/rank_first_relevant
    
    return total/number_of_queries



with open("evaluation/scores.json", "r") as file:
    scores = json.load(file)


print(f"MRR Model: {calculate_MRR('model')}")
print(f"MRR Baseline: {calculate_MRR('baseline')}")






MRR Model: 0.5
MRR Baseline: 0.6066666666666667


## MAP

In [14]:
from sklearn.metrics import average_precision_score
import numpy as np



def calculate_MAP(model):
    ratings = scores["queries"]
    number_of_queries = len(ratings)

    total = 0
    for rating in ratings:
        true_values = []
        for wine in rating[model]:
            if wine["score"] == "2" or wine["score"] == "3":
                true_values.append(1) # relevant
            else:
                true_values.append(0)

        # print(true_values)
        total += average_precision_score(true_values, [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])
    
    return total/number_of_queries

    
# our system ranked the first result best, the second result second best, ... -> [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

print(f"MAP Model: {calculate_MAP('model')}")
print(f"MAP Baseline: {calculate_MAP('baseline')}")


MAP Model: 0.4510959939531368
MAP Baseline: 0.5171111111111111


/home/iris/micromamba/envs/jupyter/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1131: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


## nDCG

In [15]:
from sklearn.metrics import ndcg_score
import numpy as np


def calculate_NDCG(model):
    ratings = scores["queries"]
    number_of_queries = len(ratings)

    true_values = []
    rank_values = []
    for rating in ratings:
        true_values_per_query = []
        for wine in rating[model]:
            if wine["score"] == None:
                continue
            true_values_per_query.append(int(wine["score"]))
        if len(true_values_per_query) != 10:
            print(f"None value detected at {rating['query']}")
            continue
        true_values.append(true_values_per_query)
        rank_values.append([1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])
    
    return ndcg_score(true_values, rank_values)
    

print(f"nDCG Model: {calculate_NDCG('model')}")
print(f"nDCG Baseline: {calculate_NDCG('baseline')}")

nDCG Model: 0.8730615632467046
nDCG Baseline: 0.8275158597358917
